# Box pendulum
- This is modelling for the rotary pendulum
- mP is the mass of the pendulum
- mA is the mass of the Arm
- $\theta$ is the angle made by the arm from the principal axis
- $\alpha$ is the angle made by the pendulum from the z axis of the arm

In [1]:
# Rotation matrices

def Rot1(th):
    """Rotation matrix from base to Arm."""
    return np.array([[np.cos(th), np.sin(th), 0],
                     [-np.sin(th), np.cos(th), 1],
                    [0, 0, 1]])

def Rot2(th):
    """Rotation matrix from Arm to Pen."""
    return np.array([
        [0, np.sin(th), -np.cos(th)],
        [0, np.cos(th),  np.sin(th)],
        [0,          0,            1]
    ])

In [2]:
# Imports
import sympy as sym
import numpy as np
from IPython.display import display #for pretty printing

In [ ]:
# create symbolic variables

# system parameters
X0,Y0, Z0 = sym.symbols(['X0','Y0', 'Z0']) # fixed position of Arm
g = sym.symbols('g')
mP,mA = sym.symbols(['mP','mA']) # mass of links
La,Lp = sym.symbols(['La','Lp']) # length of links
InA,InP = sym.symbols(['InA','InP']) # moment of intertia of links

# generalized coordinates
th,alph = sym.symbols(['\\theta','\\alpha']) #position
dth,dalph = sym.symbols(['\\dot{\\theta}','\\dot{\\alpha}']) #velocity
ddth,ddalph = sym.symbols(['\\ddot{\\theta}','\\ddot{\\alpha}']) #acceleration

q = sym.Matrix([[th],[alph]]) #group into matrices
dq = sym.Matrix([[dth],[dalph]])
ddq = sym.Matrix([[ddth],[ddalph]])

display(ddq) #display prints it as cool latex stuff

## Positions

## Velocities

### Linear velocities

In [ ]:
w1 = sym.Matrix([0, 0, dth])
v1t = w1*sym.Matrix([La, 0, 0])

display(w1, v1t)

### Angular velocities

In [ ]:
wArm = sym.Matrix([0, 0, dth])
wPen = sym.Matrix([dalph, dth*sym.sin(alph), dth*sym.cos(alph)])
display(wArm[2],dth)

## Lagrange

In [ ]:
# STEP 3: generate expressions for the kinetic and potential energy
## Kinetic energies
Thub = 0
TArm = (0.5*m1*dArm.T*dArm) + (0.5*In1*wArm.T*wArm)
TPen = (0.5*m2*dPen.T*dPen) + (0.5*In2*wPen.T*wPen)
T = TArm + TPen
display(T)
#T = sym.Matrix([0.5*m1*(dxArm**2+dyArm**2) + 0.5*m2*(dxPen**2+dyPen**2) + 0.5*In1*dth**2 + 0.5*In2*dalph**2])

## Potential energies
VArm = 0
VHub = 0
VPen = m2*g*zPen
V = sym.Matrix([VArm + VHub + VPen])
display(V)

In [ ]:
# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(1,len(q))
for i in range(len(q)):
    dT_ddq = sym.diff(T,dq[i]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = T.jacobian(q) # partial of T in q

# term 4
Lg4 = V.jacobian(q) # partial of U in q

#combine
EOM = Lg1 - Lg3 + Lg4

#display(EOM) #<-this will give you an idea of the magnitude of the faff a decent symbolic package lets you avoid

# Test

In [ ]:
#initial conditions
th0 = np.pi/4
th20 = 0
dth0 = 0
dth20 = 0

h = 0.01

#parameters
X0val = 0
Y0val = 2
Laval = 1
Lpval = 1

parameter_values = [(X0,X0val),(Y0,Y0val),(g,9.81),(m1,1),(m2,1),(La,Laval),(Lp,Lpval),(In1,0.08),(In2,0.08)]

thr = [th0]
alphr = [th20]
dthr = [dth0]
dalphr = [dth20]

N = 100
for i in range(1,N):
    # substitute parameter values and previous conditions into the EOM 
    past = [(th,thr[i-1]),(th2,alphr[i-1]),(dth,dthr[i-1]),(dth2,dalphr[i-1])]
    EOM_sub = EOM.subs(parameter_values).subs(past)
    
    # solve for the acceleration
    acc = sym.solve(EOM_sub,[ddth, ddth2])
    
    # integrate for the next velocity and position    
    dthr.append(float(dthr[i-1]+h*acc[ddth]))
    dalphr.append(float(dalphr[i-1]+h*acc[ddth2]))
    thr.append(float(thr[i-1]+h*dthr[i]))
    alphr.append(float(alphr[i-1]+h*dalphr[i]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Initialize the figure and 3D axis
fig1 = plt.figure()
ax1 = fig1.add_subplot(111, projection='3d')

# Define the lengths of the links
La = 1  # Length of the first link (arm)
Lp = 1  # Length of the second link (pendulum)

# Define the plot limits
ax1.set_xlim([-2, 2])
ax1.set_ylim([0, 4])
ax1.set_zlim([-2, 2])
ax1.set_xlabel('X axis')
ax1.set_ylabel('Y axis')
ax1.set_zlabel('Z axis')

def plot_furuta_pendulum_3d(i, th_in, th2_in, ax):
    ax.clear()
    
    # Calculate positions of the arm
    Latopx = 0
    Latopy = 0
    Latopz = 0
    
    Labottomx = La * np.cos(th_in[i])
    Labottomy = La * np.sin(th_in[i])
    Labottomz = 0
    
    # Plot the first link (arm)
    ax.plot([Latopx, Labottomx], [Latopy, Labottomy], [Latopz, Labottomz], color='k', linewidth=2)
    
    # Add axis to the arm
    ax.quiver(Latopx, Latopy, Latopz, Labottomx - Latopx, Labottomy - Latopy, Labottomz - Latopz, 
              color='r', length=La, normalize=True, arrow_length_ratio=0.1)
    
    # Calculate positions of the pendulum
    # Pendulum rotates about the end of the arm
    Lpbottomx = Labottomx + Lp * np.cos(th_in[i]) * np.cos(th2_in[i])
    Lpbottomy = Labottomy + Lp * np.sin(th_in[i]) * np.cos(th2_in[i])
    Lpbottomz = Labottomz - Lp * np.sin(th2_in[i])
    
    # Plot the second link (pendulum)
    ax.plot([Labottomx, Lpbottomx], [Labottomy, Lpbottomy], [Labottomz, Lpbottomz], color='k', linewidth=2)
    
    # Add axis to the pendulum
    ax.quiver(Labottomx, Labottomy, Labottomz, Lpbottomx - Labottomx, Lpbottomy - Labottomy, Lpbottomz - Labottomz, 
              color='b', length=Lp, normalize=True, arrow_length_ratio=0.1)

    # Set plot limits
    ax.set_xlim([-2, 2])
    ax.set_ylim([-2, 2])
    ax.set_zlim([-2, 2])
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')

# Generate some example data
N = 100  # Number of frames
thr = np.linspace(0, 2 * np.pi, N)  # Example angle data for link 1
alphr = np.linspace(0, np.pi, N)  # Example angle data for link 2

# Define the update function
update = lambda i: plot_furuta_pendulum_3d(i, thr, alphr, ax1)

# Create the animation
animate = ani.FuncAnimation(fig1, update, frames=range(N), interval=50, repeat=True)

# Convert to HTML5 video for embedding in notebook
HTML(animate.to_html5_video())


In [ ]:
#initial conditions
th0 = np.pi/4
th20 = 0
dth0 = 0
dth20 = 0

h = 0.01

#parameters
X0val = 0
Y0val = 0
Z0val = 0
Laval = 1
Lpval = 1

parameter_values = [(X0,X0val),(Y0,Y0val),(g,9.81),(m1,1),(m2,1),(La,Laval),(Lp,Lpval),(In1,0.08),(In2,0.08)]

thr = [th0]
alphr = [th20]
dthr = [dth0]
dalphr = [dth20]

N = 100
for i in range(1,N):
    # substitute parameter values and previous conditions into the EOM 
    past = [(th,thr[i-1]),(th2,alphr[i-1]),(dth,dthr[i-1]),(dth2,dalphr[i-1])]
    EOM_sub = EOM.subs(parameter_values).subs(past)
    
    # solve for the acceleration
    acc = sym.solve(EOM_sub,[ddth, ddth2])
    
    # integrate for the next velocity and position    
    dthr.append(float(dthr[i-1]+h*acc[ddth]))
    dalphr.append(float(dalphr[i-1]+h*acc[ddth2]))
    thr.append(float(thr[i-1]+h*dthr[i]))
    alphr.append(float(alphr[i-1]+h*dalphr[i]))